In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tensorflow as tf

In [9]:
print("hi")

hi


In [10]:
print("hello world")

hello world


In [11]:
print('siyung made a change')

siyung made a change
